In [1]:
# import
import pandas as pd

In [82]:
# load predictions
def read_and_store_csv(file_path):
    try:
        df = pd.read_csv(file_path, delimiter=';')
        df['datetime'] = pd.to_datetime(df['datetime'])
        df = df.sort_values(by='datetime')
        return df

    except Exception as e:
        print(f"Error reading and storing CSV: {e}")
        return None


file_path = '../data/predictions/mock_predictions.csv'
data_frame = read_and_store_csv(file_path)

if data_frame is not None:
    print(data_frame.head()) 

# load other required data
# Average Modern EVs can go 3 miles (4.8 kilometers) per kWh
avgBatteryConsumptionPerDayInKWh = 10
batteryCapacityInKWh = 70
batteryLevel= 1 # 1 = 100%
maximumChargingPowerInKW = 7.6

             datetime  Coal—PC  Gas—Combined Cycle  Biomass—cofiring  \
0 2023-11-28 00:00:00     0.15                0.10              0.05   
1 2023-11-28 01:00:00     0.16                0.12              0.04   
2 2023-11-28 02:00:00     0.14                0.11              0.06   
3 2023-11-28 03:00:00     0.13                0.13              0.05   
4 2023-11-28 04:00:00     0.12                0.15              0.04   

   Biomass—dedicated  Geothermal  Hydropower  Nuclear  \
0               0.10        0.10        0.20     0.10   
1               0.10        0.10        0.18     0.10   
2               0.12        0.10        0.17     0.11   
3               0.08        0.11        0.15     0.15   
4               0.10        0.12        0.14     0.15   

   Concentrated Solar Power  Solar PV—rooftop  Solar PV—utility  Wind onshore  \
0                      0.05              0.05              0.05          0.10   
1                      0.05              0.06              0.0

In [83]:
# calculate remaining battery level per day based on datetime
# based on that delete all rows where battery level is below 20% so that they cannot be suggested
import pandas as pd


def calculate_battery_level_per_day(dataframe, avg_battery_consumption_per_day, battery_capacity, initial_battery_level):
    df_result = dataframe.copy()
    
    df_result['batteryLevelInPercent'] = 0.0
    
    start_time = df_result['datetime'].min()
    df_result['elapsed_hours'] = (df_result['datetime'] - start_time).dt.total_seconds() / 3600
    
    avg_battery_consumption_per_hour = avg_battery_consumption_per_day / 24
    
    # Calculate the net change in battery level based on elapsed time
    for index, row in df_result.iterrows():
        
        net_change = avg_battery_consumption_per_hour * row['elapsed_hours']
        current_battery_level = max(battery_capacity * initial_battery_level - net_change, 0)
        df_result.at[index, 'batteryLevelInPercent'] = (current_battery_level / battery_capacity)
            
    return df_result

capacity_df = calculate_battery_level_per_day(data_frame, avgBatteryConsumptionPerDayInKWh, batteryCapacityInKWh, batteryLevel)

# delete all rows where battery level is below 20%
df = capacity_df[capacity_df['batteryLevelInPercent'] >= 0.2]
low_battery_df = capacity_df[capacity_df['batteryLevelInPercent'] < 0.2]
print(df)

              datetime  Coal—PC  Gas—Combined Cycle  Biomass—cofiring  \
0  2023-11-28 00:00:00     0.15                0.10              0.05   
1  2023-11-28 01:00:00     0.16                0.12              0.04   
2  2023-11-28 02:00:00     0.14                0.11              0.06   
3  2023-11-28 03:00:00     0.13                0.13              0.05   
4  2023-11-28 04:00:00     0.12                0.15              0.04   
5  2023-11-28 05:00:00     0.10                0.18              0.03   
6  2023-11-28 06:00:00     0.09                0.20              0.02   
7  2023-11-29 06:00:00     0.09                0.20              0.02   
8  2023-11-30 06:00:00     0.09                0.20              0.02   
9  2023-12-01 06:00:00     0.09                0.20              0.02   
10 2023-12-02 06:00:00     0.09                0.20              0.02   
11 2023-12-03 06:00:00     0.09                0.20              0.02   

    Biomass—dedicated  Geothermal  Hydropower  Nuc

In [84]:
# calculate relative charging emissions for each hour
emission_factors = {
    'Coal—PC': 820,
    'Gas—Combined Cycle': 490,
    'Biomass—cofiring': 740,
    'Biomass—dedicated': 230,
    'Geothermal': 38,
    'Hydropower': 24,
    'Nuclear': 12,
    'Concentrated Solar Power': 27,
    'Solar PV—rooftop': 41,
    'Solar PV—utility': 48,
    'Wind onshore': 11,
    'Wind offshore': 12
}

# Calculate total emissions for each row
df.loc[:, 'TotalEmissions'] = df.apply(lambda row: sum(row[column] * emission_factors[column] for column in df.columns[1:-4]), axis=1)


# Display the resulting DataFrame
print(df)

              datetime  Coal—PC  Gas—Combined Cycle  Biomass—cofiring  \
0  2023-11-28 00:00:00     0.15                0.10              0.05   
1  2023-11-28 01:00:00     0.16                0.12              0.04   
2  2023-11-28 02:00:00     0.14                0.11              0.06   
3  2023-11-28 03:00:00     0.13                0.13              0.05   
4  2023-11-28 04:00:00     0.12                0.15              0.04   
5  2023-11-28 05:00:00     0.10                0.18              0.03   
6  2023-11-28 06:00:00     0.09                0.20              0.02   
7  2023-11-29 06:00:00     0.09                0.20              0.02   
8  2023-11-30 06:00:00     0.09                0.20              0.02   
9  2023-12-01 06:00:00     0.09                0.20              0.02   
10 2023-12-02 06:00:00     0.09                0.20              0.02   
11 2023-12-03 06:00:00     0.09                0.20              0.02   

    Biomass—dedicated  Geothermal  Hydropower  Nuc

/var/folders/cw/0zshhmj50sxd1zjjctpchf9w0000gn/T/ipykernel_40893/951218483.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'TotalEmissions'] = df.apply(lambda row: sum(row[column] * emission_factors[column] for column in df.columns[1:-4]), axis=1)


In [85]:
# calculate charging times for starting at each hour
# mocked so far
chargingTime = pd.DataFrame()
chargingTime['datetime'] = df['datetime'].copy()
chargingTime['charging_duration'] = df['datetime'].copy().dt.hour

print(chargingTime)

              datetime  charging_duration
0  2023-11-28 00:00:00                  0
1  2023-11-28 01:00:00                  1
2  2023-11-28 02:00:00                  2
3  2023-11-28 03:00:00                  3
4  2023-11-28 04:00:00                  4
5  2023-11-28 05:00:00                  5
6  2023-11-28 06:00:00                  6
7  2023-11-29 06:00:00                  6
8  2023-11-30 06:00:00                  6
9  2023-12-01 06:00:00                  6
10 2023-12-02 06:00:00                  6
11 2023-12-03 06:00:00                  6


In [86]:
# calculate emissions for charging, starting at each hour

merged_df = pd.merge(df, chargingTime, on='datetime', how='left')

# Initialize a new column for the sum of emissions
merged_df['SumEmissionsForBeginChargingNow'] = 0.0

# Calculate the sum of emissions for each datetime along with the emissions of the following x hours
for index, row in merged_df.iterrows():
    hours_to_sum = row['charging_duration']  # Sum emissions for up to the next 6 hours
    sum_emissions = row['TotalEmissions']  # Use the TotalEmissions value

    # Add emissions of the following x hours
    for i in range(0, hours_to_sum):
        next_hour_index = index + i
        if next_hour_index < len(merged_df):
            sum_emissions += merged_df.at[next_hour_index, 'TotalEmissions']
            
    merged_df.at[index, 'SumEmissionsForBeginChargingNow'] = sum_emissions

# Display the resulting DataFrame
print(merged_df)

              datetime  Coal—PC  Gas—Combined Cycle  Biomass—cofiring  \
0  2023-11-28 00:00:00     0.15                0.10              0.05   
1  2023-11-28 01:00:00     0.16                0.12              0.04   
2  2023-11-28 02:00:00     0.14                0.11              0.06   
3  2023-11-28 03:00:00     0.13                0.13              0.05   
4  2023-11-28 04:00:00     0.12                0.15              0.04   
5  2023-11-28 05:00:00     0.10                0.18              0.03   
6  2023-11-28 06:00:00     0.09                0.20              0.02   
7  2023-11-29 06:00:00     0.09                0.20              0.02   
8  2023-11-30 06:00:00     0.09                0.20              0.02   
9  2023-12-01 06:00:00     0.09                0.20              0.02   
10 2023-12-02 06:00:00     0.09                0.20              0.02   
11 2023-12-03 06:00:00     0.09                0.20              0.02   

    Biomass—dedicated  Geothermal  Hydropower  Nuc

In [87]:
# output the top three charging times (beginning and ending) with the lowest emissions
# sort by sum of emissions
sorted_df = merged_df.sort_values(by='SumEmissionsForBeginChargingNow')
sorted_df['charging_finished'] = sorted_df['datetime'] + pd.to_timedelta(sorted_df['charging_duration'], unit='h')
print("According to the model, the best times to charge are:")
print(sorted_df[['datetime', 'charging_finished', 'charging_duration', 'SumEmissionsForBeginChargingNow']][:3])


According to the model, the best times to charge are:
              datetime   charging_finished  charging_duration  \
0  2023-11-28 00:00:00 2023-11-28 00:00:00                  0   
11 2023-12-03 06:00:00 2023-12-03 12:00:00                  6   
1  2023-11-28 01:00:00 2023-11-28 02:00:00                  1   

    SumEmissionsForBeginChargingNow  
0                            248.70  
11                           460.92  
1                            518.98  
